## Imports

In [60]:
import cvxopt
import cvxopt.glpk
from cvxopt import matrix

import numpy as np

In [61]:
# E: 0
# W: 1
# S: 2
# N: 3

d = { 'E': 0, 'W': 1, 'S': 2, 'N': 3}
n = ['E', 'W', 'S', 'N']

In [62]:
32*33*5

5280

In [63]:


def in_cross(i,j):
    return not (i in [0,1,5,6] and j in [0,1,5,6]) and i<7 and j<7 and i>=0 and j>=0

def board_to_index(i,j,t,d=None):
    # i and j from 0 to 6, t from 0 to 31 (and moves go from 0 to 30), d from 0 to 3 + 1 (state). 
    if not in_cross(i,j):
        raise Exception('Wrong indexes')
    if d is None:
        d = 4
    if j < 2: #line
        return j*3 + i + t * 33 + d * 32 * 33
    if j < 5: #line
        return 6 + (j-2) * 7 + i + t * 33 + d * 32 * 33
    else:
        return 27 + (j-5) * 3 + i + t * 33 + d * 32 * 33
    

def index_to_board(x):
    d = x // (32*33)
    x = x % (32*33)
    t = x // 33
    x = x % 33
    
    if x < 7:
        return x % 3, x // 3, t, d
    if x < 27:
        x -= 6
        return x % 7, 2 + x // 7, t, d
    else:
        x-= 27
        return x % 3, 5 + x // 3, t, d



In [64]:
print(index_to_board(5279))
print(board_to_index(2, 6, 31, 4))

(2, 6, 31, 4)
5279


In [69]:


def generate_G_and_h():
    G = []
    h = []
    for i in range(7):
        for j in range(7):
            for t in range(31):
                if in_cross(i,j):
                    #M[i, j, t, E] ≤ bState[i, j, t]
                    idx_move = board_to_index(i,j,t,d['E'])
                    idx_state = board_to_index(i,j,t)

                    row = np.zeros(33*32*5)
                    row[idx_move] = 1.0
                    row[idx_state] = -1.0
                    G.append(row)
                    h.append(0.0)

                    # M[i, j, t, E] ≤ bState[i + 1, j, t]
                    if in_cross(i+1,j):
                        idx_move = board_to_index(i,j,t,d['E'])
                        idx_state = board_to_index(i+1,j,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = -1.0
                        G.append(row)
                        h.append(0.0)

                    #M[i, j, t, E] ≤ 1 − bState[i + 2, j, t]
                    if in_cross(i+2,j):
                        idx_move = board_to_index(i,j,t,d['E'])
                        idx_state = board_to_index(i+2,j,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = 1.0
                        G.append(row)
                        h.append(1.0)

                    #M[i, j, t, W] ≤ bState[i, j, t]
                    idx_move = board_to_index(i,j,t,d['W'])
                    idx_state = board_to_index(i,j,t)

                    row = np.zeros(33*32*5)
                    row[idx_move] = 1.0
                    row[idx_state] = -1.0
                    G.append(row)
                    h.append(0.0)

                    #M[i, j, t, W ] ≤ bState[i − 1, j, t]
                    if in_cross(i-1,j):
                        idx_move = board_to_index(i,j,t,d['W'])
                        idx_state = board_to_index(i-1,j,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = -1.0
                        G.append(row)
                        h.append(0.0)
                    
                    #M[i,j,t,W]≤1−bState[i−2,j,t]
                    if in_cross(i-2,j):
                        idx_move = board_to_index(i,j,t,d['W'])
                        idx_state = board_to_index(i-2,j,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = 1.0
                        G.append(row)
                        h.append(1.0)
                    
                    #M[i, j, t, S] ≤ bState[i, j, t]
                    idx_move = board_to_index(i,j,t,d['S'])
                    idx_state = board_to_index(i,j,t)

                    row = np.zeros(33*32*5)
                    row[idx_move] = 1.0
                    row[idx_state] = -1.0
                    G.append(row)
                    h.append(0.0)
                    
                    #M[i,j,t,S] ≤ bState[i,j + 1,t]
                    if in_cross(i,j+1):
                        idx_move = board_to_index(i,j,t,d['S'])
                        idx_state = board_to_index(i,j+1,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = -1.0
                        G.append(row)
                        h.append(0.0)
                        
                    #M[i,j,t,S] ≤ 1 − bState[i,j + 2,t]
                    if in_cross(i,j+2):
                        idx_move = board_to_index(i,j,t,d['S'])
                        idx_state = board_to_index(i,j+2,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = 1.0
                        G.append(row)
                        h.append(1.0)
                    
                    #M[i, j, t, N] ≤ bState[i, j, t]
                    idx_move = board_to_index(i,j,t,d['N'])
                    idx_state = board_to_index(i,j,t)

                    row = np.zeros(33*32*5)
                    row[idx_move] = 1.0
                    row[idx_state] = -1.0
                    G.append(row)
                    h.append(0.0)
                    
                    #M[i,j,t,N] ≤ bState[i,j − 1,t]
                    if in_cross(i,j-1):
                        idx_move = board_to_index(i,j,t,d['N'])
                        idx_state = board_to_index(i,j-1,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = -1.0
                        G.append(row)
                        h.append(0.0)
                        
                    #M[i,j,t,N]≤1−bState[i,j−2,t]
                    if in_cross(i,j-2):
                        idx_move = board_to_index(i,j,t,d['N'])
                        idx_state = board_to_index(i,j-2,t)

                        row = np.zeros(33*32*5)
                        row[idx_move] = 1.0
                        row[idx_state] = 1.0
                        G.append(row)
                        h.append(1.0)
        
    return G, h

In [70]:
def generate_A_and_b():
    A = []
    
    # bState[i, j, t] − bState[i, j, t + 1]
    # - (sum for d, M[i, j, t, d])
    # - M[i − 1, j, t, E] 
    # + M[i − 2, j, t, E] 
    # - M[i + 1, j, t, W] 
    # + M[i + 2, j, t, W]
    # - M[i, j − 1, t, S] 
    # + M[i, j − 2, t, S] 
    # - M[i, j + 1, t, N] 
    # + M[i, j + 2, t, N]
    # = 0
    
    A = []
    b = []
    for i in range(7):
        for j in range(7):
            for t in range(30):
                if in_cross(i,j):
                    row = np.zeros(33*32*5)

                    row[board_to_index(i,j,t)] = 1.0
                    row[board_to_index(i,j,t+1)] = -1.0
                    
                    row[board_to_index(i,j,t,0)] = -1.0
                    row[board_to_index(i,j,t,1)] = -1.0
                    row[board_to_index(i,j,t,2)] = -1.0
                    row[board_to_index(i,j,t,3)] = -1.0
                    
                    if in_cross(i-1,j):
                        row[board_to_index(i-1,j,t,d['E'])] = -1.0
                    if in_cross(i-2,j):
                        row[board_to_index(i-2,j,t,d['E'])] = 1.0
                        
                    if in_cross(i+1,j):
                        row[board_to_index(i+1,j,t,d['W'])] = -1.0
                    if in_cross(i+2,j):
                        row[board_to_index(i+2,j,t,d['W'])] = 1.0
                        
                    if in_cross(i,j-1):
                        row[board_to_index(i,j-1,t,d['S'])] = -1.0
                    if in_cross(i,j-2):
                        row[board_to_index(i,j-2,t,d['S'])] = 1.0
                        
                    if in_cross(i,j+1):
                        row[board_to_index(i,j+1,t,d['N'])] = -1.0
                    if in_cross(i,j+2):
                        row[board_to_index(i,j+2,t,d['N'])] = 1.0
                        
                    A.append(row)
                    b.append(0.0)
    
    print(np.array(A).shape)
    
    # (M[i,j,t,E]+M[i,j,t,W]+M[i,j,t,S]+M[i,j,t,N])=1
    
    # t from 0 to 32; player must make one move from 0 to 30 (requires 31 moves to solve)
    for t in range(30):
        row = np.zeros(33*32*5) 
        for i in range(7):
            for j in range(7):
                if in_cross(i,j):              
                    row[board_to_index(i,j,t,0)] = 1.0
                    row[board_to_index(i,j,t,1)] = 1.0
                    row[board_to_index(i,j,t,2)] = 1.0
                    row[board_to_index(i,j,t,3)] = 1.0
        A.append(row)
        b.append(1.0)
    
    # last move should be equal to 0
    row_ = np.zeros(33*32*5) 
    for i in range(7):
        for j in range(7):
            if in_cross(i,j):              
                row_[board_to_index(i,j,31,0)] = 1.0
                row_[board_to_index(i,j,31,1)] = 1.0
                row_[board_to_index(i,j,31,2)] = 1.0
                row_[board_to_index(i,j,31,3)] = 1.0
    A.append(row_)
    b.append(0.0)
                    
    # intial state
    for i in range(7):
        for j in range(7):
            if in_cross(i,j):
                row = np.zeros(33*32*5)                    
                row[board_to_index(i,j,0)] = 1.0
                A.append(row)
                if i==3 and j==3:
                    b.append(0.0)
                else:
                    b.append(1.0)
                    
    # You can't just "leave" the board by the border:
    row = np.zeros(33*32*5)
    for i in range(7):
        for t in range(31):
            if in_cross(i,0):
                row[board_to_index(i,0,t, d['N'])] = 1.0
            if in_cross(i,6):
                row[board_to_index(i,6,t, d['S'])] = 1.0
            if in_cross(6,i):
                row[board_to_index(6,i,t, d['E'])] = 1.0
            if in_cross(0,i):
                row[board_to_index(0,i,t, d['W'])] = 1.0
    for i in [0,1,5,6]:
        if in_cross(4,i):
            row[board_to_index(4,i,t, d['E'])] = 1.0
        if in_cross(2,i):
            row[board_to_index(2,i,t, d['W'])] = 1.0
        if in_cross(i,4):
            row[board_to_index(i,4,t, d['S'])] = 1.0
        if in_cross(i,2):
            row[board_to_index(i,2,t, d['N'])] = 1.0   
    
    A.append(row)
    b.append(0.0)
    return A,b


def generate_c():
    c = np.zeros(33*32*5)
    idx = board_to_index(3,3,31)
    c[idx] = -1.0
    return c

In [71]:
def solve_solitary():
    A, b = generate_A_and_b()
    G, h = generate_G_and_h()
    c = generate_c()
    
    A = np.array(A)
    G = np.array(G)
    h = np.array(h)
    b = np.array(b)
    c = np.array(c)
    
    
    print(A.shape, b.shape, c.shape, G.shape, h.shape)
    
    A = matrix(A)
    G = matrix(G)
    h = matrix(h)
    b = matrix(b)
    c = matrix(c)
    
    return cvxopt.glpk.ilp(A=A, G=G, h=h, b=b, B=set([i for i in range(33*32*5)]), c=c)
    
    

In [ ]:
res = solve_solitary()

(990, 5280)


In [59]:
print(res)

('LP relaxation is primal infeasible', None)


In [11]:
c = matrix(generate_c())
cost = c.trans() * res[1]
print(cost)
print([i for i in range(len(c)) if c[i] == -1])
print(index_to_board(7815))

TypeError: unsupported operand type(s) for *: 'cvxopt.base.matrix' and 'NoneType'

In [ ]:
def convert_moves(res):
    sol = []
    
    for i in range(len(res[1])):
        if res[1][i] == 1.0:
            sol.append(index_to_board(i))
    
    sol = sorted(sol, key=lambda M : M[2])
    sol = np.array(sol)
    moves = sol[np.where(sol[:,3] < 4, True, False)]
    moves = [ (x[0], x[1], x[2], n[int(x[3])]) for x in moves ]
    return moves

def convert_state(res):
    sol = []
    
    for i in range(len(res[1])):
        if index_to_board(i)[3] == 4.0:
            sol.append(index_to_board(i) + [res[1][i]])
    
    sol = sorted(sol, key=lambda M : M[2])
    state = np.array(sol)
    state = state.tolist()    
    return state

def pegs_remaining(state):
    sol = []
    for i in range(len(state)//49):
        sol.append(sum([x[4] for x in state[i*49:i*49+49]]))
    return sol

In [ ]:
state = convert_state(res)
#pegs_remaining(state)

mods = np.where(np.array(state[0:49])[:, 4] != np.array(state[49:98])[:, 4], True, False)
print(np.array(state[0:49])[mods])
print(np.array(state[49:98])[mods])

In [ ]:
state = convert_state(res)
state[0:98]

In [ ]:
moves = convert_moves(res)
moves